# Kilter Board Data Exploration Pipeline

This notebook provides a comprehensive data exploration pipeline for analyzing SQLite database containing climbing route data.

## Table of Contents
1. [Setup and Database Connection](#setup)
2. [Database Schema Exploration](#schema)
3. [Data Quality Assessment](#quality)
4. [Exploratory Data Analysis](#eda)
5. [Statistical Analysis](#stats)
6. [Visualization Pipeline](#viz)
7. [Export and Reporting](#export)

## 1. Setup and Database Connection {#setup}

First, let's import all necessary libraries and establish connection to our SQLite database.

In [ ]:
# Core libraries
import sqlite3
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Database handling
from sqlalchemy import create_engine, text

# Jupyter widgets for interactivity
import ipywidgets as widgets
from IPython.display import display, HTML

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Configure pandas display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("Libraries imported successfully!")

In [ ]:
# Database connection setup
DATABASE_PATH = 'kilter_board_data.db'

def get_db_connection():
    """Create and return a database connection."""
    try:
        conn = sqlite3.connect(DATABASE_PATH)
        return conn
    except Exception as e:
        print(f"Error connecting to database: {e}")
        return None

def execute_query(query, params=None):
    """Execute a query and return results as a pandas DataFrame."""
    conn = get_db_connection()
    if conn:
        try:
            df = pd.read_sql_query(query, conn, params=params)
            conn.close()
            return df
        except Exception as e:
            print(f"Error executing query: {e}")
            conn.close()
            return None
    return None

# Test connection
test_conn = get_db_connection()
if test_conn:
    print("✅ Database connection successful!")
    test_conn.close()
else:
    print("❌ Database connection failed. Make sure to run 'python create_sample_db.py' first.")

## 2. Database Schema Exploration {#schema}

Let's explore the database structure and understand our data model.

In [ ]:
# Get list of all tables
tables_query = """
SELECT name FROM sqlite_master 
WHERE type='table' AND name NOT LIKE 'sqlite_%'
ORDER BY name;
"""

tables_df = execute_query(tables_query)
print("Database Tables:")
print("=" * 20)
for table in tables_df['name']:
    print(f"📋 {table}")

In [ ]:
# Get schema information for each table
def get_table_schema(table_name):
    """Get schema information for a specific table."""
    schema_query = f"PRAGMA table_info({table_name})"
    return execute_query(schema_query)

# Display schema for all tables
for table in tables_df['name']:
    print(f"\n🔍 Schema for '{table}' table:")
    schema = get_table_schema(table)
    print(schema.to_string(index=False))
    
    # Get row count
    count_query = f"SELECT COUNT(*) as row_count FROM {table}"
    count_result = execute_query(count_query)
    print(f"📊 Total rows: {count_result['row_count'].iloc[0]}")
    print("-" * 50)

## 3. Data Quality Assessment {#quality}

Let's assess the quality of our data by checking for missing values, duplicates, and data consistency.

In [ ]:
# Load all tables into DataFrames for analysis
holds_df = execute_query("SELECT * FROM holds")
routes_df = execute_query("SELECT * FROM routes")
route_holds_df = execute_query("SELECT * FROM route_holds")
attempts_df = execute_query("SELECT * FROM user_attempts")

# Create a dictionary for easy access
dataframes = {
    'holds': holds_df,
    'routes': routes_df,
    'route_holds': route_holds_df,
    'user_attempts': attempts_df
}

print("✅ All tables loaded into DataFrames")

In [ ]:
# Data quality assessment function
def assess_data_quality(df, table_name):
    """Assess data quality for a given DataFrame."""
    print(f"\n📊 Data Quality Report for '{table_name}' table:")
    print("=" * 50)
    
    # Basic info
    print(f"Shape: {df.shape}")
    print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024:.2f} KB")
    
    # Missing values
    missing_values = df.isnull().sum()
    if missing_values.sum() > 0:
        print("\n❌ Missing Values:")
        for col, count in missing_values[missing_values > 0].items():
            percentage = (count / len(df)) * 100
            print(f"  {col}: {count} ({percentage:.2f}%)")
    else:
        print("\n✅ No missing values found")
    
    # Duplicate rows
    duplicates = df.duplicated().sum()
    if duplicates > 0:
        print(f"\n❌ Duplicate rows: {duplicates}")
    else:
        print("\n✅ No duplicate rows found")
    
    # Data types
    print("\n📋 Data Types:")
    for col, dtype in df.dtypes.items():
        print(f"  {col}: {dtype}")
    
    return {
        'shape': df.shape,
        'missing_values': missing_values.sum(),
        'duplicates': duplicates,
        'memory_usage_kb': df.memory_usage(deep=True).sum() / 1024
    }

# Assess quality for all tables
quality_reports = {}
for table_name, df in dataframes.items():
    quality_reports[table_name] = assess_data_quality(df, table_name)

In [ ]:
# Summary quality dashboard
quality_summary = pd.DataFrame(quality_reports).T
quality_summary.columns = ['Shape', 'Missing Values', 'Duplicates', 'Memory (KB)']
quality_summary['Rows'] = quality_summary['Shape'].apply(lambda x: x[0])
quality_summary['Columns'] = quality_summary['Shape'].apply(lambda x: x[1])
quality_summary = quality_summary[['Rows', 'Columns', 'Missing Values', 'Duplicates', 'Memory (KB)']]

print("\n📋 Data Quality Summary:")
print("=" * 30)
print(quality_summary.to_string())

## 4. Exploratory Data Analysis {#eda}

Now let's dive into exploring our data with descriptive statistics and initial insights.

In [ ]:
# Descriptive statistics for numerical columns
print("📊 Descriptive Statistics for Holds:")
print("=" * 40)
print(holds_df.describe())

print("\n📊 Descriptive Statistics for Routes:")
print("=" * 40)
print(routes_df.describe())

print("\n📊 Descriptive Statistics for User Attempts:")
print("=" * 40)
print(attempts_df.describe())

In [ ]:
# Categorical analysis
print("🏷️ Hold Types Distribution:")
hold_types = holds_df['hold_type'].value_counts()
print(hold_types)

print("\n🎯 Grade Distribution:")
grade_dist = routes_df['grade'].value_counts().sort_index()
print(grade_dist)

print("\n👤 Setter Distribution:")
setter_dist = routes_df['setter_name'].value_counts()
print(setter_dist)

print("\n🎪 Attempt Results Distribution:")
attempt_results = attempts_df['attempt_result'].value_counts()
print(attempt_results)

In [ ]:
# Advanced queries for insights
insights_queries = {
    'route_difficulty_by_setter': """
        SELECT setter_name, AVG(grade_numeric) as avg_difficulty, COUNT(*) as routes_set
        FROM routes 
        GROUP BY setter_name 
        ORDER BY avg_difficulty DESC
    """,
    
    'success_rate_by_grade': """
        SELECT r.grade, 
               COUNT(CASE WHEN ua.attempt_result IN ('flash', 'send') THEN 1 END) as successful_attempts,
               COUNT(*) as total_attempts,
               ROUND(COUNT(CASE WHEN ua.attempt_result IN ('flash', 'send') THEN 1 END) * 100.0 / COUNT(*), 2) as success_rate
        FROM user_attempts ua
        JOIN routes r ON ua.route_id = r.id
        GROUP BY r.grade
        ORDER BY r.grade_numeric
    """,
    
    'most_popular_routes': """
        SELECT r.name, r.grade, COUNT(*) as attempt_count,
               ROUND(AVG(ua.difficulty_rating), 2) as avg_user_rating
        FROM user_attempts ua
        JOIN routes r ON ua.route_id = r.id
        GROUP BY r.id, r.name, r.grade
        ORDER BY attempt_count DESC
        LIMIT 10
    """,
    
    'hold_usage_stats': """
        SELECT h.hold_type, COUNT(*) as usage_count, AVG(h.difficulty_contribution) as avg_difficulty
        FROM holds h
        JOIN route_holds rh ON h.id = rh.hold_id
        GROUP BY h.hold_type
        ORDER BY usage_count DESC
    """
}

# Execute insight queries
insights = {}
for query_name, query in insights_queries.items():
    result = execute_query(query)
    insights[query_name] = result
    print(f"\n🔍 {query_name.replace('_', ' ').title()}:")
    print("=" * 50)
    print(result.to_string(index=False))

## 5. Statistical Analysis {#stats}

Let's perform some statistical analysis to understand relationships and correlations in our data.

In [ ]:
# Correlation analysis
numerical_cols = ['grade_numeric', 'difficulty_rating', 'attempts_count']

# Create a comprehensive dataset for correlation analysis
analysis_df = execute_query("""
    SELECT 
        r.grade_numeric,
        ua.difficulty_rating,
        ua.attempts_count,
        CASE WHEN ua.attempt_result IN ('flash', 'send') THEN 1 ELSE 0 END as success,
        ua.user_id,
        r.id as route_id
    FROM user_attempts ua
    JOIN routes r ON ua.route_id = r.id
""")

print("📊 Correlation Matrix:")
correlation_matrix = analysis_df[numerical_cols].corr()
print(correlation_matrix)

# Statistical tests
from scipy import stats

print("\n🧮 Statistical Tests:")
print("=" * 30)

# Test correlation between grade and user difficulty rating
correlation, p_value = stats.pearsonr(analysis_df['grade_numeric'], analysis_df['difficulty_rating'])
print(f"Grade vs User Rating Correlation: {correlation:.3f} (p-value: {p_value:.3f})")

# Test correlation between grade and attempts needed
correlation, p_value = stats.pearsonr(analysis_df['grade_numeric'], analysis_df['attempts_count'])
print(f"Grade vs Attempts Correlation: {correlation:.3f} (p-value: {p_value:.3f})")

In [ ]:
# User performance analysis
user_stats = execute_query("""
    SELECT 
        user_id,
        COUNT(*) as total_attempts,
        COUNT(CASE WHEN attempt_result IN ('flash', 'send') THEN 1 END) as successful_attempts,
        ROUND(COUNT(CASE WHEN attempt_result IN ('flash', 'send') THEN 1 END) * 100.0 / COUNT(*), 2) as success_rate,
        AVG(difficulty_rating) as avg_perceived_difficulty,
        MAX(r.grade_numeric) as hardest_grade_attempted,
        AVG(attempts_count) as avg_attempts_per_route
    FROM user_attempts ua
    JOIN routes r ON ua.route_id = r.id
    GROUP BY user_id
    HAVING total_attempts >= 10
    ORDER BY success_rate DESC
""")

print("👥 User Performance Statistics:")
print("=" * 40)
print(user_stats.head(10).to_string(index=False))

print(f"\n📈 Performance Summary:")
print(f"Average Success Rate: {user_stats['success_rate'].mean():.2f}%")
print(f"Median Success Rate: {user_stats['success_rate'].median():.2f}%")
print(f"Best Performer: User {user_stats.iloc[0]['user_id']} ({user_stats.iloc[0]['success_rate']:.2f}% success rate)")

## 6. Visualization Pipeline {#viz}

Now let's create comprehensive visualizations to better understand our data.

In [ ]:
# Set up the plotting environment
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Kilter Board Data Overview', fontsize=16, fontweight='bold')

# Plot 1: Hold types distribution
hold_types.plot(kind='bar', ax=axes[0,0], color='skyblue')
axes[0,0].set_title('Distribution of Hold Types')
axes[0,0].set_xlabel('Hold Type')
axes[0,0].set_ylabel('Count')
axes[0,0].tick_params(axis='x', rotation=45)

# Plot 2: Grade distribution
grade_dist.plot(kind='bar', ax=axes[0,1], color='lightcoral')
axes[0,1].set_title('Distribution of Route Grades')
axes[0,1].set_xlabel('Grade')
axes[0,1].set_ylabel('Count')
axes[0,1].tick_params(axis='x', rotation=45)

# Plot 3: Attempt results
attempt_results.plot(kind='pie', ax=axes[1,0], autopct='%1.1f%%')
axes[1,0].set_title('Attempt Results Distribution')
axes[1,0].set_ylabel('')

# Plot 4: Success rate by grade
success_by_grade = insights['success_rate_by_grade']
axes[1,1].bar(success_by_grade['grade'], success_by_grade['success_rate'], color='lightgreen')
axes[1,1].set_title('Success Rate by Grade')
axes[1,1].set_xlabel('Grade')
axes[1,1].set_ylabel('Success Rate (%)')
axes[1,1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Interactive Plotly visualizations
# 1. Hold positions on the board
fig_holds = px.scatter(holds_df, x='position_x', y='position_y', 
                      color='hold_type', size='difficulty_contribution',
                      title='Hold Positions on Kilter Board',
                      labels={'position_x': 'X Position (%)', 'position_y': 'Y Position (%)'},
                      width=800, height=600)
fig_holds.update_layout(showlegend=True)
fig_holds.show()

# 2. User performance heatmap
user_grade_performance = execute_query("""
    SELECT 
        ua.user_id,
        r.grade,
        COUNT(CASE WHEN ua.attempt_result IN ('flash', 'send') THEN 1 END) as successes,
        COUNT(*) as attempts
    FROM user_attempts ua
    JOIN routes r ON ua.route_id = r.id
    GROUP BY ua.user_id, r.grade
    HAVING attempts >= 3
""")
user_grade_performance['success_rate'] = user_grade_performance['successes'] / user_grade_performance['attempts']

# Create pivot table for heatmap
heatmap_data = user_grade_performance.pivot(index='user_id', columns='grade', values='success_rate')
heatmap_data = heatmap_data.fillna(0)

fig_heatmap = px.imshow(heatmap_data, 
                       title='User Success Rate by Grade',
                       labels={'x': 'Grade', 'y': 'User ID', 'color': 'Success Rate'},
                       aspect='auto',
                       color_continuous_scale='RdYlGn')
fig_heatmap.show()

In [ ]:
# Advanced time series analysis
# Convert timestamp to datetime
attempts_df['attempted_at'] = pd.to_datetime(attempts_df['attempted_at'])
attempts_df['date'] = attempts_df['attempted_at'].dt.date
attempts_df['month'] = attempts_df['attempted_at'].dt.to_period('M')

# Daily activity
daily_activity = attempts_df.groupby('date').size().reset_index(name='attempts')
daily_activity['date'] = pd.to_datetime(daily_activity['date'])

fig_time = px.line(daily_activity, x='date', y='attempts',
                  title='Daily Climbing Activity Over Time',
                  labels={'date': 'Date', 'attempts': 'Number of Attempts'})
fig_time.show()

# Monthly success rate trends
monthly_stats = attempts_df.groupby('month').agg({
    'attempt_result': lambda x: (x.isin(['flash', 'send'])).sum(),
    'id': 'count'
}).reset_index()
monthly_stats.columns = ['month', 'successes', 'total_attempts']
monthly_stats['success_rate'] = monthly_stats['successes'] / monthly_stats['total_attempts']
monthly_stats['month_str'] = monthly_stats['month'].astype(str)

fig_monthly = px.bar(monthly_stats, x='month_str', y='success_rate',
                    title='Monthly Success Rate Trends',
                    labels={'month_str': 'Month', 'success_rate': 'Success Rate'})
fig_monthly.show()

In [ ]:
# Interactive dashboard with widgets
def create_interactive_analysis():
    """Create interactive widgets for data exploration."""
    
    # Widget for grade selection
    grade_widget = widgets.SelectMultiple(
        options=list(routes_df['grade'].unique()),
        value=list(routes_df['grade'].unique())[:3],
        description='Grades:',
        disabled=False
    )
    
    # Widget for hold type selection
    hold_widget = widgets.SelectMultiple(
        options=list(holds_df['hold_type'].unique()),
        value=list(holds_df['hold_type'].unique()),
        description='Hold Types:',
        disabled=False
    )
    
    def update_analysis(selected_grades, selected_holds):
        """Update analysis based on widget selections."""
        filtered_query = f"""
            SELECT r.grade, COUNT(*) as route_count, AVG(ua.difficulty_rating) as avg_rating
            FROM routes r
            LEFT JOIN user_attempts ua ON r.id = ua.route_id
            WHERE r.grade IN ({','.join([f"'{g}'" for g in selected_grades])})
            GROUP BY r.grade
            ORDER BY r.grade_numeric
        """
        
        result = execute_query(filtered_query)
        
        if result is not None and not result.empty:
            fig = px.bar(result, x='grade', y='route_count',
                        title=f'Route Count for Selected Grades',
                        color='avg_rating')
            fig.show()
    
    # Create interactive widget
    interactive_plot = widgets.interactive(update_analysis, 
                                         selected_grades=grade_widget,
                                         selected_holds=hold_widget)
    
    display(interactive_plot)

# Create the interactive dashboard
create_interactive_analysis()

## 7. Export and Reporting {#export}

Finally, let's create export capabilities and generate summary reports.

In [ ]:
# Export functions
def export_summary_report():
    """Generate and export a comprehensive summary report."""
    
    report = {
        'database_overview': {
            'total_holds': len(holds_df),
            'total_routes': len(routes_df),
            'total_attempts': len(attempts_df),
            'unique_users': attempts_df['user_id'].nunique(),
            'date_range': f"{attempts_df['attempted_at'].min()} to {attempts_df['attempted_at'].max()}"
        },
        'route_statistics': {
            'grade_distribution': routes_df['grade'].value_counts().to_dict(),
            'setter_distribution': routes_df['setter_name'].value_counts().to_dict(),
            'average_difficulty': routes_df['grade_numeric'].mean()
        },
        'performance_metrics': {
            'overall_success_rate': len(attempts_df[attempts_df['attempt_result'].isin(['flash', 'send'])]) / len(attempts_df) * 100,
            'average_attempts_per_route': attempts_df['attempts_count'].mean(),
            'most_difficult_grade': routes_df.loc[routes_df['grade_numeric'].idxmax(), 'grade'],
            'most_popular_route': insights['most_popular_routes'].iloc[0]['name']
        },
        'data_quality': quality_reports
    }
    
    return report

# Generate the report
summary_report = export_summary_report()

print("📋 KILTER BOARD DATA ANALYSIS SUMMARY REPORT")
print("=" * 55)

print("\n🗄️ Database Overview:")
for key, value in summary_report['database_overview'].items():
    print(f"  {key.replace('_', ' ').title()}: {value}")

print("\n📊 Route Statistics:")
print(f"  Average Difficulty: {summary_report['route_statistics']['average_difficulty']:.2f}")
print(f"  Most Common Grade: {max(summary_report['route_statistics']['grade_distribution'], key=summary_report['route_statistics']['grade_distribution'].get)}")
print(f"  Most Active Setter: {max(summary_report['route_statistics']['setter_distribution'], key=summary_report['route_statistics']['setter_distribution'].get)}")

print("\n🎯 Performance Metrics:")
for key, value in summary_report['performance_metrics'].items():
    if isinstance(value, float):
        print(f"  {key.replace('_', ' ').title()}: {value:.2f}")
    else:
        print(f"  {key.replace('_', ' ').title()}: {value}")

In [ ]:
# Export data to CSV files
def export_to_csv():
    """Export processed data to CSV files for further analysis."""
    
    # Create exports directory
    import os
    os.makedirs('exports', exist_ok=True)
    
    # Export main datasets
    holds_df.to_csv('exports/holds_data.csv', index=False)
    routes_df.to_csv('exports/routes_data.csv', index=False)
    attempts_df.to_csv('exports/attempts_data.csv', index=False)
    
    # Export analysis results
    user_stats.to_csv('exports/user_performance_stats.csv', index=False)
    
    for name, df in insights.items():
        df.to_csv(f'exports/{name}.csv', index=False)
    
    print("✅ Data exported to CSV files in 'exports/' directory")
    print("Files created:")
    for file in os.listdir('exports'):
        print(f"  📄 {file}")

# Export the data
export_to_csv()

In [ ]:
# Save report as JSON for programmatic access
import json

with open('exports/analysis_report.json', 'w') as f:
    json.dump(summary_report, f, indent=2, default=str)

print("✅ Complete analysis report saved as 'exports/analysis_report.json'")

# Display final summary
print("\n🎉 DATA EXPLORATION PIPELINE COMPLETED SUCCESSFULLY!")
print("=" * 55)
print("✅ Database connection established and tested")
print("✅ Schema exploration completed")
print("✅ Data quality assessment performed")
print("✅ Exploratory data analysis conducted")
print("✅ Statistical analysis completed")
print("✅ Comprehensive visualizations created")
print("✅ Data exported for further analysis")
print("✅ Summary report generated")
print("\n📁 All results saved in 'exports/' directory")
print("\n🔬 Ready for advanced analysis and modeling!")